In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install rasterio
!pip install tensorboardX
!pip install boto3
!pip install S3Fs
!pip install urllib3==1.25.5

In [2]:
import os
import sys
import importlib
from pathlib import Path
import pandas as pd

In [ ]:
pd.show_versions()

In [ ]:
git_username = "agroimpacts"
git_token = "<scrub>"
repo = "deeplearner"
branch_name = "normalization_test"
clone_path = "/content/gdrive/MyDrive/sam/FieldBoundaryDataset/FB_working_folder"

if not os.path.exists(clone_path):
    os.makedirs(clone_path)

git_path = f'https://{git_token}@github.com/{git_username}/{repo}.git'
repo_clone_path = f"{clone_path}/{repo}"

os.chdir(clone_path)
if not os.path.isdir(repo_clone_path):
  !git clone -b "{branch_name}" "{git_path}"
else:
  if os.listdir(repo_clone_path):
    os.chdir(repo_clone_path)
    !git branch
    !git config core.fileMode false  # needed to ignore permissions changes
    !git pull

os.chdir("..")

In [ ]:
sys.path.insert(0, os.path.join(repo_clone_path, 'deeplearner/'))
sys.path.insert(0, repo_clone_path)
import deeplearner
importlib.reload(deeplearner)
from deeplearner.models import *
from deeplearner.losses import *
from deeplearner.datatorch2 import *
from deeplearner.utils import *
from deeplearner.compiler import *

In [ ]:
config = {
    "source_dir" : "/content/gdrive/MyDrive/sam/FieldBoundaryDataset/source",
    "working_dir" : "/content/gdrive/MyDrive/sam/FieldBoundaryDataset/FB_working_folder",
    
    # train and validation dataset
    "train_csv_name" : "catalog_gh_cg_tz_ng_v1.csv",
    "train_pickle_name" : "train_local_per_tile.pickle",
    "val_pickle_name" : "val_local_per_tile.pickle",
    "lbl_patchSize" : 200,
    "one_side_buffer" : 12,
    "tile_buffer" : 11,
    "img_path_cols" : ["dir_os"],
    "norm_stats_type" : "local_per_tile",
    "label_path_col" : "dir_label",
    "train_lbl_quality_groups" : (0, 2, 3, 4),
    "val_lbl_quality_groups" : (3, 4),
    "transformations" : ['vflip', 'hflip', 'rotate', 'resize', 'shift_brightness'],
    "rotationDegree" : (-90, 90),
    "bshift_band_grouping" : [4],

    # train and validation DataLoader
    "train_BatchSize" : 20,
    "val_BatchSize" : 2,

    # Model
    "input_channels" : 4,
    "n_classes" : 3,

    # Model compiler
    "gpuDevices" : [0],
    "params_init_path" : None,
    "freeze_layer_ls" : None,
    

    # Model fitting
    "epochs" : 5,
    "optimizer": "nesterov",
    "LR" : 0.01, 
    "LR_policy" : "PolynomialLR",
    "criterion" : "BalancedTverskyFocalLoss(gamma = 0.9)",
    "momentum" : 0.95,
    "resume" : False,
    "resume_epoch" : None,
    "bucket" : "activemapper",
    "prefix_out": "/content/gdrive/MyDrive/sam/FieldBoundaryDataset/FB_working_folder/ex1_local_per_tile",

    #prediction 
}

pickle_dir = Path(config["source_dir"]) / "pickles"
if not os.path.exists(pickle_dir):
    os.makedirs(pickle_dir)

train_pickle_path = pickle_dir / config["train_pickle_name"]
val_pickle_path = pickle_dir / config["val_pickle_name"]

if not os.path.exists(config["working_dir"]):
    os.makedirs(config["working_dir"])

log_dir = Path(config["working_dir"]) / "logs"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# create train and val datasets and pickle them.
Only need to be run once if the pickled datasets do not exist.

In [ ]:
train_catalog = pd.read_csv(os.path.join(config["source_dir"], 
                                         config["train_csv_name"]))
train_catalog

In [ ]:
# load train dataset
train_dataset = planetData(dataPath = config["source_dir"],
                           catalog = train_catalog,
                           log_dir = log_dir, 
                           dataSize = config["lbl_patchSize"], 
                           buffer = config["one_side_buffer"],
                           bufferComp = config["tile_buffer"],  
                           usage = "train", 
                           imgPathCols = config["img_path_cols"],
                           norm_stats_type = config["norm_stats_type"], 
                           labelPathCol = config["label_path_col"], 
                           labelGroup = config["train_lbl_quality_groups"], 
                           deRotate = config["rotationDegree"], 
                           bShiftSubs = config["bshift_band_grouping"], 
                           trans=config["transformations"])

In [ ]:
pickle_dataset(train_dataset, train_pickle_path)

In [ ]:
validation_dataset = planetData(dataPath = config["source_dir"],
                                log_dir = log_dir,
                                catalog = train_catalog, 
                                dataSize = config["lbl_patchSize"], 
                                buffer = config["one_side_buffer"],
                                bufferComp = config["tile_buffer"],
                                usage = "validate", 
                                imgPathCols = config["img_path_cols"],
                                labelPathCol = config["label_path_col"], 
                                labelGroup=config["val_lbl_quality_groups"])

In [ ]:
pickle_dataset(validation_dataset, val_pickle_path)

# Walking through the workflow step by step

**Step 1.** Setup seeding to make the experiment reproducible. 

In [ ]:
make_reproducible()

**Step 2.** Load the train and val datasets (i.e. divide the dataset into mini-batches after applying the augmentation, convert to tensor and put them on GPU if available)

In [ ]:
train_dataset = load_dataset(train_pickle_path)
train_dataloader = DataLoader(train_dataset, 
                              batch_size=config["train_BatchSize"], 
                              shuffle=True)

In [ ]:
validation_dataset = load_dataset(val_pickle_path)
validation_dataloader = DataLoader(validation_dataset, 
                                 batch_size=config["val_BatchSize"], 
                                 shuffle=False)

**Step 3.** Initialize the model

In [ ]:
model = eval('unet'.lower())(config["input_channels"], config["n_classes"])

**Step 4.** Compile

The model Compiler is responsible for: 
1) handling the model parallelism on multiple GPUs, 
2) loading existing model parametrs if needed and
3) freeze user-defined layers of model if model-based transfer learning is pursued.

In [ ]:
model = ModelCompiler(model, 
                      buffer = config["one_side_buffer"], 
                      gpuDevices = config["gpuDevices"], 
                      params_init = config["params_init_path"],
                      freeze_params = config["freeze_layer_ls"])

**Step 5.** train the model

In [ ]:
model.fit(train_dataloader, 
          validation_dataloader, 
          epochs = config["epochs"], 
          optimizer_name = config["optimizer"], 
          lr_init = config["LR"], 
          lr_policy = config["LR_policy"], 
          criterion = config["criterion"], 
          momentum = config["momentum"],
          resume = config["resume"], 
          resume_epoch = config["resume_epoch"])

**Step 6.** Save the trained parameters

In [ ]:
model.save(bucket=config["bucket"], outPrefix=config["prefix_out"])

**Step 7.** Evaluate the trained model against the evaluation dataset and report a number of accuracy metrics in a csv format.

In [ ]:
model.evaluate(validate_dataloader, bucket=config["bucket"], prefix_out=config["prefix_out"])

**Step 8.** Prediction

In [ ]:
def load_pred_data(dir_data, pred_patch_size, pred_buffer, pred_composite_buffer, 
                   pred_batch, catalog, catalog_row, img_path_cols, average_neighbors=False):
    def load_single_tile(catalog_ind = catalog_row):
        dataset = planetData(dir_data, catalog, pred_patch_size, pred_buffer, 
                             pred_composite_buffer, "predict", 
                             catalogIndex=catalog_ind, imgPathCols=img_path_cols)
        data_loader = DataLoader(dataset, batch_size=pred_batch, shuffle=False)
        meta = dataset.meta
        tile = dataset.tile
        return data_loader, meta, tile

    if average_neighbors == True:
        catalog["tile_col_row"] = catalog.apply(lambda x: "{}_{}".format(x['tile_col'], x['tile_row']), axis=1)
        tile_col = catalog.iloc[catalog_row].tile_col
        tile_row = catalog.iloc[catalog_row].tile_row
        row_dict = {
            "center": catalog_row,
            "top": catalog.query('tile_col=={} & tile_row=={}'.format(tile_col, tile_row - 1)).iloc[0].name \
                if "{}_{}".format(tile_col, tile_row - 1) in list(catalog.tile_col_row) else None,
            "left" : catalog.query('tile_col=={} & tile_row=={}'.format(tile_col - 1, tile_row)).iloc[0].name \
                if "{}_{}".format(tile_col - 1, tile_row) in list(catalog.tile_col_row) else None,
            "right" : catalog.query('tile_col=={} & tile_row=={}'.format(tile_col + 1, tile_row)).iloc[0].name \
                if "{}_{}".format(tile_col + 1, tile_row) in list(catalog.tile_col_row) else None,
            "bottom": catalog.query('tile_col=={} & tile_row=={}'.format(tile_col, tile_row + 1)).iloc[0].name \
                if "{}_{}".format(tile_col, tile_row + 1) in list(catalog.tile_col_row) else None,
            }
        dataset_dict = {k:load_single_tile(catalog_ind = row_dict[k]) if row_dict[k] is not None else None 
                        for k in row_dict.keys()}
        return dataset_dict
    # direct crop edge pixels
    else:
        return load_single_tile()

In [ ]:
prefix_out = 'DL/predictions/gh_cg_tz/DFUNet_WithoutAttention_05032022/ghana'
catalog = 'catalogs/predict/catalog_ghana_retiled_8.csv'

In [ ]:
pred_catalog = pd.read_csv(os.path.join(dir_data, catalog))
inds = pred_catalog.query("type == 'center'").index.values

In [ ]:
for i in inds:
    print("Predicting on index %s" % (i))
    pred_dataloader = load_pred_data(
        dir_data, pred_patch_size, pred_buffer, pred_composite_buffer, 
        pred_batch, pred_catalog, i, img_path_cols, 
        average_neighbors = average_neighbors
    )
    p = model.predict(
        pred_dataloader, bucket, prefix_out, 
        pred_buffer, averageNeighbors=average_neighbors, 
        shrinkBuffer = shrink_pixels
    )

In [ ]:
import math
fig, ax = pyplot.subplots(3,3, figsize=(15,20))
for i in range(len(inds)):
    col, row = pred_catalog.iloc[inds[i]][['tile_col', 'tile_row']]
    image_path = 's3://{}/{}/Score_1/score_c{}_r{}.tif'.\
        format(bucket, prefix_out, col, row)
    img = rasterio.open(image_path).read()
    show(img.astype('uint8'), ax = ax[math.floor(i/3)][i%3], title=os.path.basename(image_path))
    
image_path
pyplot.savefig("score_map_unet.png")

In [ ]:
import math
import re
from rasterio.plot import reshape_as_raster, reshape_as_image

def findRowCol(filename):
    col = re.findall(r"(?<=_c)\d\d\d(?=_r)", filename)[0]
    row = re.findall(r"(?<=_r)\d\d\d(?=.tif)", filename)[0]
    return int(row), int(col)

def rescale_image(image, bands=(0, 1, 2, 3)):
    img = reshape_as_image(image.read())[:,:,bands]
    max_vals = [img[:, :, band].max() for band in range(img.shape[-1])]
    img = img.astype('float64')
    for band in bands:
        band_vals = img[:, :, band]
        img[:, :, band] = band_vals / max_vals[band]

    return img

path_to_unet_tiles = 'predicted_tiles_unet'
path_to_dfunet_noattn_tiles = 'predicted_tiles_dfunet_noattn'
path_to_simpledfunet_tiles = 'predicted_tiles_simpledfunet'

predicted_unet = os.listdir(path_to_unet_tiles)
predicted_dfunet = os.listdir(path_to_dfunet_noattn_tiles)
predicted_simpledfunet = os.listdir(path_to_simpledfunet_tiles)

assert(predicted_dfunet == predicted_unet == predicted_simpledfunet)
n = len(predicted_unet)
assert(len(os.listdir('dir_os')) == len(os.listdir('dir_gs')) == n)

fig, ax = pyplot.subplots(n, 4, figsize=(18,40))

os_images = []
for i in range(n):
    unet_path = os.path.join(path_to_unet_tiles, predicted_unet[i])
    img = rasterio.open(unet_path).read()
    show(img.astype('uint8'), ax = ax[i][0], title="UNet_"+os.path.basename(unet_path)[:-4])
    
    dfunet_path = os.path.join(path_to_dfunet_noattn_tiles, predicted_unet[i])
    img = rasterio.open(dfunet_path).read()
    show(img.astype('uint8'), ax = ax[i][1], title="DFUNet_NoAttn_"+os.path.basename(unet_path)[:-4])
    
    simpledfunet_path = os.path.join(path_to_simpledfunet_tiles, predicted_unet[i])
    img = rasterio.open(simpledfunet_path).read()
    show(img.astype('uint8'), ax = ax[i][2], title="Simple DFUNet_"+os.path.basename(unet_path)[:-4])
    
    row, col = findRowCol(unet_path)
    # gs_path = os.path.basename(pred_catalog.loc[pred_catalog['tile_col']==col, 'dir_gs'].iloc[0])
    # img = rasterio.open(f"dir_gs/{gs_path}").read()
    # show(img.astype('uint8'), ax = ax[i][3], title="GS")
    
    os_path = os.path.basename(pred_catalog.loc[pred_catalog['tile_col']==col, 'dir_os'].iloc[0])
    img = rasterio.open(f"dir_os/{os_path}")
    os_images.append(img)
    ax_curr = ax[i][3]
    img_rescaled = rescale_image(img)
    ax_curr.imshow(img_rescaled[:,:,(3,2,1)])

pyplot.savefig("score_map_compare_simpledfunet.svg",facecolor='white', format='svg',transparent=False, dpi=300)